In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00


In [1]:
import sys
print("Python version:", sys.version)

Python version: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [ ]:
import time

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification

In [ ]:
#df loading
df_train = pd.read_csv('train.csv')[['sentence','hate speech']]
df_val = pd.read_csv('val.csv')[['sentence','hate speech']]
df_test = pd.read_csv('test.csv')[['sentence','hate speech']]
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)
df_train

(40224, 2)
(5028, 2)
(5029, 2)


,sentence,hate speech
0,.... ঐ ইন্দুর তোই মরছ নাই?,1
1,#গেবনের শেষে আইসা আপনার মুখোশ টা খুলছে এতেই আম...,1
2,✈✈✈✈��� মুরগি চোরের পাছায় ডুকবি আর মারবি।।।,1
3,১৮ কোটির চোদা খাওয়া শেষে এখন ১৫০কোটির চোদা খাব...,1
4,২য় মীর জাফরের মুখে মুতে দে...,1
...,...,...
40219,��� গানটা চালু করে ওয়াস রুমে গিয়েছিলাম! এসে দে...,0
40220,���� হালকার উপজেলা ঝাপসা,0
40221,���� ��,0
40222,���� ��পুরাই আগুনের গুলা� () ��,0


In [ ]:
#determining the max length of sentence
max_len_sentence=0
df_test['sentence'].str.len().idxmax()
df_test['sentence'][472]

'বাংলাদেশ অাওয়ামীলীগের চার বারের সফল সাধারণ সম্পাদকভাষা সৈনিকমুক্তিযোদ্ধা মরহুম রাস্ট্রপতি ও শহীদ অাইভী রহমানের সুযোগ্য সন্তান নাজমুল হাসান পাপন।ছোট বেলা থেকেই অত্যন্ত মেধাবী নাজমুল হাসান পাপন ঢাকা বিশ্ববিদ্যালয়ের অাইবিএ থেকে মাস্টার্স ডিগ্রি সম্পন্ন করেন। অামেরিকার ইউনিভার্সিটি অব ক্যালোর্ফোনিয়া এবং নর্থ ওয়েস্টার্ন ইউনিভার্সিটি সিকাগো থেকে এক্সিকিউটিভ এডুকেশন এন্ড মার্কেটিং স্ট্রেটেজি উপর ডিগ্রি লাভ করেন। সালে যোগদান করেন বেক্সিমকো ফার্মায়। তারপর থেকে নিজ যোগ্যতায় বাংলাদেশের ঔষধ শিল্পকে নিয়ে গেছেন এক অনন্য উচ্চতায়।দায়িত্ব পালন করছেন বাংলাদেশ ঔষধ শিল্প সমিতির সভাপতি হিসেবে। তার নেতৃত্বে বেক্সিমকো প্রথম বর্হিবিশ্বে বাংলাদেশে তৈরী ঔষধ রপ্তানি করে।সারা পৃথিবীতে তাকে স্বাস্থ্যসেবা ও ঔষধ নীতিমালা বিশেষজ্ঞ হিসেবে বিবেচনা করা হয়। লন্ডনের কমনওয়েলথ সচিবালয়ের একজন পরামর্শক তিনি।ঢাকা বিশ্ববিদ্যালয়ের অাইবিএর এলামনি এসোসিয়েশন সভাপতি।অসাধারণ ব্যবসায়িক নেতৃত্বেগুনের জন্য দুবাই এ অনুষ্ঠিত এশিয়ার সবচেয়ে প্রভাবশালী ব্যবসায়িক ব্যক্তিত্ব পুরুষ্কার এশিয়ান ব্র্যান্ড সামিট ঘোষিত হন নাজমুল হাসান পাপন। অামরিকা

In [ ]:
#defining some hyperparameters
max_number_input_tokens=256
batch_size_training = 16
first_dropout_rate = 0.3
hidden_output = 768
bert_model_name = "sagorsarker/bangla-bert-base"
adam_opt_lr = 3e-5
scheduler_step = 1
scheduler_gamma = 0.8
epochs = 10
classes = 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DirPath = ('/content/drive/My Drive/Test/')

Mounted at /content/drive


In [ ]:
class NewsDatasets(Dataset):
    def __init__(self, data, max_length=max_number_input_tokens):
        self.data = data

        self.config = {
            "max_length": max_length,
            "padding": "max_length",
            "return_tensors": "pt",
            "truncation": True,
            "add_special_tokens": True,
            "truncation_strategy":"longest_first"
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        value = self.data.iloc[idx]
        return value['sentence'] , value['hate speech']

In [ ]:
training_data = NewsDatasets(df_train)
train_dataloader = DataLoader(training_data, batch_size=batch_size_training, shuffle=True)

val_data = NewsDatasets(df_val)
val_dataloader = DataLoader(val_data, batch_size=batch_size_training, shuffle=True)

test_data = NewsDatasets(df_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size_training, shuffle=True)

In [ ]:
class HateSpeechBert(nn.Module):

    def __init__(self, bert):
        super(HateSpeechBert, self).__init__()

        self.bert = bert

        # dropout layer
        self.dropout = nn.Dropout(first_dropout_rate)

        # relu activation function
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()

        # dense layer 1
        self.fc1 = nn.Linear(hidden_output*2, hidden_output)

        #dense layer 2
        self.fc2 = nn.Linear(hidden_output, 128)

        # dense layer 2 (Output layer)
        self.fc3 = nn.Linear(128, 2)

        #softmax
        self.softmax = nn.Softmax(dim=1)

    # define the forward pass
    def forward(self, input_ids, token_type_ids, attention_mask):
        # pass the inputs to the model
        out = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        mean, _ = torch.max(out[0], 1)
        x= torch.cat((mean,out[1]), dim=1)

        x = self.dropout(x)

        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        # output layer
        x = self.fc3(x)
        x = self.softmax(x)

        return x

In [ ]:
class BERTBengali(nn.Module):
    def __init__(self, bert):
        super(BERTBengali, self).__init__()
        #self.bert = BertForMaskedLM.from_pretrained("sagorsarker/bangla-bert-base")
        self.bert = bert
        self.bert_drop = nn.Dropout(0.2)
        self.out = nn.Linear(hidden_output, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        bo = self.bert_drop(output[1])

        output = self.out(bo)
        return output

In [ ]:
class BERTBengaliPooler(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliPooler, self).__init__()
        self.bert = bert
        #self.bert.pooler.dense = nn.Linear(bert.config.hidden_size, bert.config.hidden_size)
        self.bert_drop = nn.Dropout(first_dropout_rate)
        self.out = nn.Linear(bert.config.hidden_size, classes)
        #softmax
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs.pooler_output
        bo = self.bert_drop(pooled_output)

        output = self.out(bo)
        output = self.softmax(bo)
        return output

In [ ]:
class CustomBERTBengali(nn.Module):
    def __init__(self, bert):
        super(CustomBERTBengali, self).__init__()
        self.bert = bert
        self.bert_drop = nn.Dropout(first_dropout_rate)
        self.tanh = nn.Tanh()
        self.out = nn.Linear(hidden_output * 3, classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        o1 = outputs.hidden_states[-1]
        o2 = outputs.pooler_output
        apool = torch.mean(o1, 1)
        mpool, _ = torch.max(o1, 1)
        pooled_output = o2
        cat = torch.cat((apool, mpool, pooled_output), 1)
        bo = self.bert_drop(cat)
        logits = self.out(bo)
        logits = self.softmax(logits)
        return logits

In [ ]:
class BERTBengaliTwo(nn.Module):
    def __init__(self, bert):
        super(BERTBengaliTwo, self).__init__()
        self.bert = bert
        self.drop_out = nn.Dropout(first_dropout_rate)
        self.l0 =  nn.Linear(hidden_output * 2, classes)
        torch.nn.init.normal_(self.l0.weight, std=0.02)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        out = torch.cat((outputs.hidden_states[-1], outputs.hidden_states[-2]), dim=-1)
        out = self.drop_out(out)
        out = out[:,0,:]
        logits = self.l0(out)
        logits = self.softmax(logits)
        return logits

In [ ]:
bert = BertModel.from_pretrained(bert_model_name, output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CustomBERTBengali(bert)
model.to(device)

CustomBERTBengali(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(102025, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
s = "আমি বাংলায় গান গাই। [SEP]"
t = tokenizer.encode_plus(s, return_tensors="pt").to(device)
print(t)
out = model(**t)
print(out)

{'input_ids': tensor([[  101,  2169,  2492,  9294,  2552, 13985,  1014,   102,   102]],
       device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
tensor([[0.5213, 0.4787]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [ ]:
from torch.optim.lr_scheduler import StepLR

optimizer = AdamW(model.parameters(), lr=adam_opt_lr)
criterion = nn.CrossEntropyLoss()
scheduler = StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

In [ ]:
def train(model, dataloader, optimizer, criterion, config):
    model.train()  # prep model for training
    train_loss = 0
    for batch in tqdm(dataloader):
        text, labels = batch

        model.zero_grad()

        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        #labels = labels.to(device)
        labels = labels.to(device, dtype=torch.long)  # Convert labels to torch.long

        # move things to model
        logs = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss = criterion(logs, labels)
        train_loss += loss.item() * input_ids.size(0)
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    return train_loss

In [ ]:
def evaluate(model, dataloader, criterion, config):
    total = 0
    correct = 0
    valid_loss = 0.0

    model.eval()  # prep model for evaluation
    for batch in dataloader:
        text, labels = batch
        inputs = tokenizer.batch_encode_plus(
            text, **config
        )
        input_ids = inputs['input_ids'].to(device)
        token_type_ids = inputs['token_type_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        labels = labels.to(device)

        # move things to model
        output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)

        loss_p = criterion(output, labels)
        # update running validation loss
        valid_loss += loss_p.item() * input_ids.size(0)
        # calculate accuracy
        proba = torch.exp(output)
        top_p, top_class = proba.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        # accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total, correct, valid_loss

In [ ]:

tokenizer_config = {
    "max_length": max_number_input_tokens,
    "padding": "max_length",
    "return_tensors": "pt",
    "truncation": True,
    "add_special_tokens": True,
     "truncation_strategy":"longest_first"
}

In [ ]:
train_loss_data, valid_loss_data = [], []
valid_loss_min = np.Inf
since = time.time()
best_loss = np.inf

for epoch in range(epochs):
    print("Epoch: {}/{}".format(epoch + 1, epochs))
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    total = 0
    correct = 0
    e_since = time.time()

    # Train Model
    train_loss += train(model, train_dataloader, optimizer, criterion, tokenizer_config)
    # Now Evaluate
    out = evaluate(model, val_dataloader, criterion, tokenizer_config)
    total += out[0]
    correct += out[1]
    valid_loss += out[2]

    scheduler.step()

    # print training/validation statistics
    # calculate average loss over an epoch
    train_loss = train_loss / len(train_dataloader.dataset)
    valid_loss = valid_loss / len(val_dataloader.dataset)

    # calculate train loss and running loss
    train_loss_data.append(train_loss * 100)
    valid_loss_data.append(valid_loss * 100)

    if valid_loss < best_loss:
        best_loss = valid_loss
        torch.save(model.state_dict(), DirPath+bert_model_name+"_modeltest.pth")
        print(f'epoch: {epoch+1}')

    print("\tTrain loss:{:.6f}..".format(train_loss),
          "\tValid Loss:{:.6f}..".format(valid_loss),
          "\tVal Accuracy: {:.4f}".format(correct / total * 100))

time_elapsed = time.time() - since
print('Training completed in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

Epoch: 1/10


  0%|          | 0/2514 [00:00<?, ?it/s]

epoch: 1
	Train loss:0.459852.. 	Valid Loss:0.437362.. 	Val Accuracy: 87.5696
Epoch: 2/10


  0%|          | 0/2514 [00:00<?, ?it/s]

epoch: 2
	Train loss:0.432054.. 	Valid Loss:0.431820.. 	Val Accuracy: 88.1464
Epoch: 3/10


...

  0%|          | 0/2514 [00:00<?, ?it/s]

epoch: 6
	Train loss:0.421220.. 	Valid Loss:0.428216.. 	Val Accuracy: 88.5044
Epoch: 7/10


  0%|          | 0/2514 [00:00<?, ?it/s]

	Train loss:0.426529.. 	Valid Loss:0.432955.. 	Val Accuracy: 88.0270
Epoch: 8/10


  0%|          | 0/2514 [00:00<?, ?it/s]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(train_loss_data, label="Training loss")
plt.plot(valid_loss_data, label="validation loss")
plt.legend(frameon=False)

In [ ]:
model.load_state_dict(torch.load(DirPath+bert_model_name+"_modeltest.pth", map_location = device))

In [ ]:
all_preds = []
all_labels = []

for batch in test_dataloader:
    text, labels = batch
    inputs = tokenizer.batch_encode_plus(
        text, **tokenizer_config
    )
    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    labels = labels.to(device)

    # move things to model
    output = model(token_type_ids=token_type_ids, input_ids=input_ids, attention_mask=attention_mask)
    preds = output.detach().cpu().numpy()
    preds = np.argmax(preds, axis = 1)
    all_preds.extend(preds)
    all_labels.extend(labels.cpu().numpy())

In [ ]:
from sklearn.metrics import classification_report

# preds = np.argmax(preds, axis = 1)
print(classification_report(all_labels, all_preds))